Data Type: microarray data with fluorescence intensity 

# Step 1: delete mean values, use median value for fluorescense intensity

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from os import path

Input: raw data file

In [4]:
# mac
rawdata_file = path.join('..','data','raw_data','/Users/louxuwen/Desktop/Documents/GitHub/modulome-C_Glutamicum_Microarray_clean/1_metadata_file/raw_data.xlsx') # Enter metadata filename here
# linux
#rawdata_file = path.join('..','data','raw_data','/home/amy/Documents/GitHub/modulome-C_Glutamicum_Microarray_clean/1_metadata_file/raw_data.xlsx') # Enter metadata filename here

In [5]:
raw_data = pd.read_excel(rawdata_file,index_col=0).fillna(0);
print('Number of genes:',raw_data.shape[1])
print('Number of experiments:',raw_data.shape[0])

Number of genes: 3050
Number of experiments: 4200


select median value only to avoid effect from extreme values

In [6]:
raw_data

,Mean/Median,Fluorescence,Label strain,cg0001,cg0002,cg0004,cg0005,cg0006,cg0007,cg0008,...,cg3433,cg3434,cg4000,cg4001,cg4002,cg4003,cg4004,cg4005,cg4006,cg4007
Sample name,,,,,,,,,,,,,,,,,,,,,
∆6C (0.75 µM mitomycin C) vs. WT (0.75 µM mitomycin C)-rep1,Mean,B635,Cy3,544.853,0,570.844,364,660,353.797,0,...,0,864,0,0,0,0,0,664.81,0,786
∆6C (0.75 µM mitomycin C) vs. WT (0.75 µM mitomycin C)-rep1,Mean,F635,Cy3,2911.36,0,8385.39,2183,2155,6630.57,0,...,0,4707,0,0,0,0,0,13202.2,0,1412
∆6C (0.75 µM mitomycin C) vs. WT (0.75 µM mitomycin C)-rep1,Mean,B635,Cy5,1039.64,0,1174.1,704,1398,501.299,0,...,0,1843,0,0,0,0,0,1242.36,0,1338
∆6C (0.75 µM mitomycin C) vs. WT (0.75 µM mitomycin C)-rep1,Mean,F635,Cy5,1413.79,0,4514.29,1141,1008,1961.85,0,...,0,2492,0,0,0,0,0,3260.48,0,1762
∆6C (0.75 µM mitomycin C) vs. WT (0.75 µM mitomycin C)-rep1,Median,B635,Cy5,552.159,0,611.196,282,579,183.016,0,...,0,567,0,0,0,0,0,740.225,0,444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WT PnrdH::Ptuf PctaE::Ptuf PctaC::Ptuf vs. WT-rep3,Median,B635,Cy3,56.921,0,60,56,0,51.4782,0,...,0,49.8397,51,0,0,53.8516,52,54.3139,0,0
WT PnrdH::Ptuf PctaE::Ptuf PctaC::Ptuf vs. WT-rep3,Median,F635,Cy3,473.018,0,1384,234,0,1178.22,0,...,0,161.555,1323,0,0,478.046,195,477.661,0,0
WT PnrdH::Ptuf PctaE::Ptuf PctaC::Ptuf vs. WT-rep3,Mean,B635,Cy5,51.4393,0,51,49,0,46.4973,0,...,0,44.4297,45,0,0,47.9062,47,47.833,0,0


In [7]:
median_data = raw_data.loc[raw_data['Mean/Median'] == "Median"]

In [8]:
print('Number of genes:',median_data.shape[1])
print('Number of experiments:',median_data.shape[0])

Number of genes: 3050
Number of experiments: 2100


In [11]:
from pathlib import Path  
filepath = Path('/Users/louxuwen/Desktop/Documents/GitHub/modulome-C_Glutamicum_Microarray_clean/1_metadata_file/median_data.xlsx')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
median_data.to_excel(filepath)  

In [ ]:
b_data = median_data.loc[median_data['Fluorescence'] == "B635"]
f_data = median_data.loc[median_data['Fluorescence'] == "F635"]

In [ ]:
b_data_v = b_data.iloc[:,3:]
f_data_v = f_data.iloc[:,3:]

In [ ]:
b_data_v.columns